In [ ]:
#Python library for OrientDB
import pyorient as py

%run main.ipynb

In [ ]:
#create connection
class DBCreator:
    def  __init__(self,db_name):
        self.db_name = db_name
        
    def create(self):
        client = py.OrientDB('localhost',2424)
        session_id = client.connect( "root", "root" )

        #create a databse 
        #client.db_create(self.db_name,py.DB_TYPE_GRAPH,py.STORAGE_TYPE_PLOCAL)
        return client

#Class for opening of DB
class OpenDB:
    def  __init__(self,client,db_name):
        #open databse 
        client.db_open( db_name, "root", "root" ) 

#Class for creation of table
class CreateTable:
    #create class
    def  __init__(self,client,table_name,parent_table,attribute_list):
        self.client = client
        cluster_id = self.client.command( "create class "+ str(table_name)+" extends " + str(parent_table)) 
        for att in attribute_list:
            #create property 
            att_name,att_type = att
            cluster_id = self.client.command( "create property "+ str(table_name)+"."+str(att_name)+" "+str(att_type)) 

#Class for insert query
class InsertTuple:
    def __init__(self,client,table_name,value_list):
        #insert record
        query = "insert into "+str(table_name)+" ("
        for val in value_list:
            val_name,val_value = val
            query=query+str(val_name)+","
        leng = len(query)
        query = query[0:leng-1]
        query=query+") values ("
        for val in value_list:
            val_name,val_value = val
            query=query+"'"+str(val_value)+"' ,"
        leng = len(query)
        query = query[0:leng-1]
        query=query+")"
        client.command(query) 

#Class for select query
class SelectQuery:
    def __init__(self,client):
        self.client = client
        
    
    def get_data(self,table_name):
        data = self.client.query("SELECT FROM " + str(table_name), 400)
        return data

In [ ]:
#Create DB
creator = DBCreator("mainDB")
client = creator.create()

In [ ]:
#Open DB
dbOpener = OpenDB(client,"mainDB")

In [ ]:
#Column list
att_list = [('id','Integer'),('name','String')] 

In [ ]:
att_list


[('id', 'Integer'), ('name', 'String')]

In [ ]:
#Table creation
tableCreator = CreateTable(client,"users","V",att_list)

In [ ]:
#Reading SVNIT twitter handles from file 
users_list = read_from_file("twitter_handles")
#print(users_list)
#ct = 1
#for user in users_list:
#    print(user)
#    val_list=[('id',ct),('name',user)]
#    print(val_list)
#    InsertTuple(client,"users",val_list)
#    ct=ct+1

In [ ]:
users_list = users_list[192:]
print(users_list[0])

In [ ]:
twitter_client = TwitterClient()
twitter_analyzer = TweetAnalyzer()

select_query = SelectQuery(client)
users_data = select_query.get_data('users')


#Getting identifiers of each user from OrientDB
users_dict = {}
for data in users_data:
    tmp = vars(data)
    name = tmp['_OrientRecord__o_storage']['name']
    rid = tmp['_OrientRecord__rid']
    users_dict.update({name:rid})
print(users_dict)

In [ ]:
twitter_client = TwitterClient()
twitter_analyzer = TweetAnalyzer()

#Edge Creation between users and followers
for user in users_list:
    print(user)
    twitter_client_user = TwitterClient(user)
    current_user_follower_list = twitter_analyzer.tweets_to_list(twitter_client_user.get_followers())
    for follower in current_user_follower_list:
        if follower in users_list:
            client.command("CREATE EDGE FROM " + str(users_dict[user]) + " TO " + str(users_dict[follower]))
    

In [ ]:
user="Maulikchevli"
twitter_client_user = TwitterClient(user)
current_user_follower_list = twitter_analyzer.tweets_to_list(twitter_client_user.get_followers())
print(current_user_follower_list)